Creating variable for year ranges to be scraped and for url of page to be scraped. notice the {} brackets in the url. We can use that to interate through the list of years to be scraped

In [1]:
years = list(range(1991,2022))

In [2]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

Next we use a for loop to interate every year in the years list and extract the page for that year from the url. Then with the open() function, we save the page as a html file

In [3]:
import requests         #importing library

for year in years:
    url = url_start.format(year)   
    data = requests.get(url)        #getting html data for each year in years list
    
    with open("new_mvp/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

We import and use the BeautifulSoup library to parse through the html pages

In [4]:
from bs4 import BeautifulSoup

In [5]:
import pandas as pd

In [6]:
mvp_df = []    # empty list to hold dataframe of data for each year
for year in years:   #looping through years list
    with open("new_mvp/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")  #initializing beautiful soup
    soup.find('tr', class_="over_header").decompose() # removing unwanted row from html
    mvp_table = soup.find(id='mvp')  # selecting table in html using id
    mvp = pd.read_html(str(mvp_table))[0]    #converting html to pandas dataframe
    mvp["Year"] = year  #adding year column for identification
    
    mvp_df.append(mvp)  #append to mvp_df 

In [7]:
mvps = pd.concat(mvp_df) # concating into one dataframe with pandas

In [8]:
mvps.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [9]:
mvps.to_csv("mvps.csv")  #write dataframe to .csv file

Now we collect player stats for all the players for each year using Selenium Library

In [10]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [11]:
from selenium import webdriver

Install driver for chrome broswer

In [17]:
driver = webdriver.Chrome(executable_path="C:/Users/DELL/Downloads/chromedriver_win32/chromedriver")

C:\Users\DELL\AppData\Local\Temp/ipykernel_13460/1439656237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/DELL/Downloads/chromedriver_win32/chromedriver")


In [18]:
import time
for year in years:
    url = player_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,1000)")
    time.sleep(2)
    data = driver.page_source
    
    with open("player_stats/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data)

WebDriverException: Message: target frame detached
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x005C9943+2595139]
	Ordinal0 [0x0055C9F1+2148849]
	Ordinal0 [0x004543F0+1065968]
	Ordinal0 [0x00445A97+1006231]
	Ordinal0 [0x00444AD0+1002192]
	Ordinal0 [0x004450D8+1003736]
	Ordinal0 [0x00445068+1003624]
	Ordinal0 [0x0044B220+1028640]
	Ordinal0 [0x004462AD+1008301]
	Ordinal0 [0x004467C5+1009605]
	Ordinal0 [0x004465AF+1009071]
	Ordinal0 [0x00445BC6+1006534]
	Ordinal0 [0x0044542B+1004587]
	Ordinal0 [0x004452C9+1004233]
	Ordinal0 [0x004558A0+1071264]
	Ordinal0 [0x004AB3BB+1422267]
	Ordinal0 [0x0049B806+1357830]
	Ordinal0 [0x00476086+1204358]
	Ordinal0 [0x00476F96+1208214]
	GetHandleVerifier [0x0076B232+1658114]
	GetHandleVerifier [0x0082312C+2411516]
	GetHandleVerifier [0x0065F261+560433]
	GetHandleVerifier [0x0065E366+556598]
	Ordinal0 [0x0056286B+2173035]
	Ordinal0 [0x005675F8+2192888]
	Ordinal0 [0x005676E5+2193125]
	Ordinal0 [0x005711FC+2232828]
	BaseThreadInitTh

In [19]:
player_stats_dfs = []
for year in years:
    with open("player_stats/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find(id='per_game_stats')
    player = pd.read_html(str(player_table))[0]
    player["Year"] = year
    
    player_stats_dfs.append(player) 

In [20]:
player = pd.concat(player_stats_dfs)

In [21]:
player

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,...,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,...,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,...,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,...,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,536,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,...,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0,2021
726,537,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,...,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1,2021
727,538,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,...,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3,2021
728,539,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,...,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4,2021


In [22]:
player.to_csv("players.csv")

Getting Team stats

In [23]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}.html"

In [24]:
for year in years:
    url = team_stats_url.format(year)
    data = requests.get(url)
    
    with open("team_stats/{}.html".format(year), "w+", encoding="utf-8") as f:
              f.write(data.text)

In [25]:
team_stats_df = []
for year in years:
    with open("team_stats/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
        
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id='divs_standings_E')
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    team_stats_df.append(team) 
    
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id='divs_standings_W')
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    team_stats_df.append(team)

In [26]:
teams = pd.concat(team_stats_df)

In [27]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*
14,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*
15,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
16,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


In [28]:
teams.to_csv("teams.csv")